In [1]:
from itertools import islice
from collections import defaultdict


In [13]:
def read_file(fn):
    with open(fn, 'r') as f:
        yield from map(str.strip, f)

def read_search_table(fn):
    yield from map(lambda x: x.split('\t')[:2], read_file(fn))
    
    
def get_systems(fn):
    systems = defaultdict(lambda: defaultdict(set))
    for subject, target in read_search_table(fn):
        system, component = subject.split('-')
        systems[system][component].add(target)
    return systems

def get_protein_id_from_gff_info(info):
    """ Returns the protein_id from a gff info field """
    return dict(map(lambda x: x.split('='), info.split(';'))).get('locus_tag')

def parse_merged_gff(fn):
    """ Parses a merged gff file and returns a generator of tuples """

    lines = filter(lambda x: not x.startswith('#'), read_file(fn))
    while (line := next(lines, None)):
        contig, *_, info = line.split('\t')
        assembly, contig = contig.split('_')
        protein_id = get_protein_id_from_gff_info(info)
        yield assembly, contig, protein_id

def mmseqs_to_dict(fn: str) -> dict:
    return dict(map(lambda x: x.strip().split('\t'), read_file(fn)))



In [152]:
def search_matches(merged_gff, search_table, cluster_map):
    """ Search matches in the gff file and returns a generator of tuples """
    systems = get_systems(search_table)

    cluster_map = mmseqs_to_dict(cluster_map)

    current_assembly_contig = None
    for assembly, contig, protein_id in parse_merged_gff(merged_gff):
        if current_assembly_contig != (assembly, contig):
            current_assembly_contig = (assembly, contig)
            current_index = 0

        current_index += 1
        as_cluster = cluster_map.get(protein_id, 'NA')

        for system, components in systems.items():
            for component, targets in components.items():
                if as_cluster in targets:
                    yield (assembly, contig, current_index, system, component, as_cluster)
    

def get_simple_frequency(df, total_assemblies):
    return (
        df
        .loc[:, ['assembly', 'system', 'component']]
        .drop_duplicates()
        .groupby(['system', 'component'])
        .size()
        .apply(lambda x: x / total_assemblies)
        .rename('frequency')
        .to_frame()
    )



def get_windows(diff, window_size):
    """ Returns a dataframe with the windows for each system """
    result = []

    for i in diff:
        if not result:
            slide = 1
            result.append(slide)
            continue
            
        if i <= window_size:
            result.append(slide)
            continue
            
        slide += 1
        result.append(slide)

    return result

def get_colloc_freq(df, window_size, system_components, total_assemblies):
    return (
        df
        .groupby(['assembly', 'contig', 'system'],  group_keys=False)
        .apply(
            lambda df_: (
                df_.assign(
                    window=lambda df: get_windows(df.on_contig_index.diff().fillna(False), window_size)
                )
            )
        )
        .loc[:, ['assembly', 'contig', 'system', 'component', 'window']]
        .groupby(['assembly', 'contig', 'system', 'window'])
        .agg(set)
        .reset_index()
        .assign(
            is_complete = lambda df: df[['system','component']].apply(lambda x: x[1] == system_components[x[0]], axis=1)
        )
        .loc[:, ['assembly', 'system', 'is_complete']]
        .groupby(['assembly', 'system'])
        .any()
        .reset_index()
        .loc[lambda df: df.is_complete, ['assembly', 'system']]
        .groupby('system')
        .size()
        .rename('frequency')
        .apply(lambda x: x / total_assemblies)
        .to_frame()
    )

In [22]:
import pandas as pd

data = search_matches(
    "../../results/gff/all.cds.gff.sample.out",
    '/home/hugo/projects/icetea/results/mmseqs/New-DS-V2_X_SALMONELLA_REPS.search.tsv',
    "../../results/mmseqs/CLUSTERS_REPS_unique.tsv"
)

systems_matches = pd.DataFrame(data, columns=['assembly', 'contig', 'on_contig_index', 'system', 'component', 'cluster'])


total_assemblies = 112

systems_matches

,assembly,contig,on_contig_index,system,component,cluster
0,590.14785,7,82,DS2,1,GKCJFLAD_04304
1,85569.336,22,27,DS3,1,FDOCFBME_04669
2,59201.1672,18,8,DS4,2,PGPIBDBP_02126
3,59201.1672,18,9,DS4,1,PLOGOCIG_03756
4,90371.1187,1,2755,DS3,1,FDOCFBME_04669
...,...,...,...,...,...,...
59,90105.169,42,309,DS1,1,HCKDJKIF_02548
60,90105.169,42,310,DS1,2,OECGJKMM_02137
61,90105.169,44,36,DS3,1,FDOCFBME_04669
62,90105.165,12,307,DS1,1,HCKDJKIF_02548


In [153]:
window_size = 5
system_components = dict(map(lambda x: (x[0], set(x[1].keys())), get_systems('/home/hugo/projects/icetea/results/mmseqs/New-DS-V2_X_SALMONELLA_REPS.search.tsv').items()))

get_colloc_freq(systems_matches, window_size, system_components, total_assemblies)

,frequency
system,
DS2,0.026786
DS3,0.258929
DS4,0.062500


False

In [111]:
# test window approach
# input test data
t = (56, 58, 90, 93, 96)

# desired output for window_size = 5
# (1, 1, 2)


(
    pd.DataFrame(
        t,
        columns=['on_contig_index']
    )
    .assign(
        window=lambda df: df.on_contig_index.diff()
    )
)

,on_contig_index,window
0,56,NaN
1,58,2.0
2,90,32.0
3,93,3.0
4,96,3.0


In [159]:
def get_protein_id_from_gff_info(info):
    """ Returns the protein_id from a gff info field """
    return dict(map(lambda x: x.split('='), info.split(';'))).get('protein_id', 'NA')

def parse_merged_gff(fn: str):
    """ Parses a merged gff file and returns a generator of tuples """

    lines = read_file(fn)

    while (line := next(lines, None)):
        if line.startswith('#!genome-build-accession NCBI_Assembly:'):
            assembly = line.split(':')[-1]
            continue

        if line.startswith('#'):
            continue

        contig, *_, info = line.split('\t')
        yield assembly, contig, get_protein_id_from_gff_info(info)


def test(merged_gff):
    for assembly, contigs, protein_id in parse_merged_gff(merged_gff):
        print(assembly, contigs, protein_id)

test('/home/hugo/projects/icetea/results/gff/refseq.gff')
        

GCF_004635825.1 NZ_CP026904.1 WP_023853434.1
GCF_004635825.1 NZ_CP026904.1 WP_023853413.1
GCF_004635825.1 NZ_CP026904.1 WP_023994834.1
GCF_004635825.1 NZ_CP026904.1 WP_124740660.1
GCF_004635825.1 NZ_CP026904.1 WP_153302771.1
GCF_004635825.1 NZ_CP026904.1 WP_019247355.1
GCF_004635825.1 NZ_CP026904.1 WP_023994624.1
GCF_004635825.1 NZ_CP026904.1 WP_019247482.1
GCF_004635825.1 NZ_CP026904.1 WP_162096758.1
GCF_004635825.1 NZ_CP026904.1 WP_023852951.1
GCF_004635825.1 NZ_CP026904.1 WP_162291062.1
GCF_004635825.1 NZ_CP026904.1 WP_033456131.1
GCF_004635825.1 NZ_CP026904.1 WP_023853503.1
GCF_004635825.1 NZ_CP026904.1 WP_224019407.1
GCF_004635825.1 NZ_CP026904.1 WP_170954289.1
GCF_004635825.1 NZ_CP026904.1 WP_169507376.1
GCF_004635825.1 NZ_CP026904.1 WP_033455115.1
GCF_004635825.1 NZ_CP026904.1 WP_124740709.1
GCF_004635825.1 NZ_CP026904.1 WP_162268681.1
GCF_004635825.1 NZ_CP026904.1 WP_019248938.1
GCF_004635825.1 NZ_CP026904.1 WP_050817833.1
GCF_004635825.1 NZ_CP026904.1 WP_019248843.1
GCF_004635

ValueError: too many values to unpack (expected 2)